<a href="https://colab.research.google.com/github/niravmistry09/face-detection-/blob/main/face_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install face-recognition opencv-python numpy


In [2]:
import face_recognition
import numpy as np
import cv2
import os
from datetime import datetime
from google.colab.output import eval_js
from base64 import b64decode
from IPython.display import display, Javascript
import pytz

In [79]:
path = '/content/Training_Images'
images = []
classNames = []
myList = os.listdir(path)

In [80]:
myList

['hardik_pandya.jpg',
 'guru_randhawa.jpg',
 'mukesh_ambani.jpg',
 'shahrukh_khan.jpg',
 'rahul_gandhi.jpg',
 'nirav_mistry.jpeg',
 'virat_kohli.jpg',
 'Narendra_Modi.jpg',
 'neeraj_chopra.jpg',
 'salman_khan.jpg',
 'abdul_kalam.jpg']

In [81]:
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)

    name = os.path.splitext(cl)[0]
    classNames.append(name)

print(f"Total Images Found: {len(images)}")
print(f"Names: {classNames}")

Total Images Found: 11
Names: ['hardik_pandya', 'guru_randhawa', 'mukesh_ambani', 'shahrukh_khan', 'rahul_gandhi', 'nirav_mistry', 'virat_kohli', 'Narendra_Modi', 'neeraj_chopra', 'salman_khan', 'abdul_kalam']


In [82]:
def findEncodings(images):
    encodeList = []

    for img in images:
        # BGR2RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # 2. Chehre ke 128 unique points (encoding) nikalna
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)

    return encodeList

encodeListKnown = findEncodings(images)
print('Encoding Complete!')

Encoding Complete!


In [7]:
!pip install pytz

In [83]:
def markAttendance(name):
    with open('Attendance.csv', 'r+') as f:
        myDataList = f.readlines()
        nameList = []

        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])

        # 2. Check karna: Agar naam list mein nahi hai, tabhi entry karna
        if name not in nameList:
            IST = pytz.timezone('Asia/Kolkata')
            now = datetime.now(IST)
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')
            print(f"Attendance Marked for: {name}")

In [84]:
from IPython.display import Javascript, display
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
import face_recognition

# def get_frame_and_close():
#     js = Javascript('''
#         async function getFrame() {
#             // 1. Camera start karna
#             const stream = await navigator.mediaDevices.getUserMedia({video: true});
#             const video = document.createElement('video');
#             video.srcObject = stream;
#             await video.play();

#             // 2. Sirf ek frame capture karna
#             const canvas = document.createElement('canvas');
#             canvas.width = 640; canvas.height = 480;
#             canvas.getContext('2d').drawImage(video, 0, 0);

#             // 3. Sabse important: Hardware tracks ko turant band karna
#             stream.getTracks().forEach(track => track.stop());
#             video.remove();

#             return canvas.toDataURL('image/jpeg', 0.8);
#         }
#     ''')
#     display(js)
#     return eval_js('getFrame()')


# print("Scanning... (Camera light will turn off after detection)")

try:
    found = False
    while not found:
        # Camera on -> Capture -> Camera off (In one go)
        data = get_frame_and_close()

        img_bytes = b64decode(data.split(',')[1])
        img = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), 1)
        imgS = cv2.cvtColor(cv2.resize(img, (0,0), None, 0.25, 0.25), cv2.COLOR_BGR2RGB)

        facesCurFrame = face_recognition.face_locations(imgS)
        encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

        for encodeFace in encodesCurFrame:
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
            matchIndex = np.argmin(faceDis)

            if matches[matchIndex] and faceDis[matchIndex] < 0.6:
                name = classNames[matchIndex].upper()
                print(f"✅ Recognized: {name}. Closing System.")
                markAttendance(name)
                found = True
                break

        if not found:
            print("No match, retrying...")

except Exception as e:
    print("Stopped:", e)

<IPython.core.display.Javascript object>

✅ Recognized: NIRAV_MISTRY. Closing System.


In [88]:
import cv2
import face_recognition
import numpy as np

# 1. Test image ka path (Jo image aapne upload ki hai)
test_image_path = '/content/hardik_pandya.jpg' # Yahan apni file ka naam likhein

# 2. Image load karein
test_img = cv2.imread(test_image_path)
test_img_rgb = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)

# 3. Test image mein face dhundna aur encode karna
faces_in_frame = face_recognition.face_locations(test_img_rgb)
encodes_in_frame = face_recognition.face_encodings(test_img_rgb, faces_in_frame)

# 4. Matching Logic
for encodeFace, faceLoc in zip(encodes_in_frame, faces_in_frame):
    # Purani 'encodeListKnown' se compare karna
    matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
    faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)

    matchIndex = np.argmin(faceDis)

    if matches[matchIndex]:
        name = classNames[matchIndex].upper()
        print(f"Match Found: {name} (Confidence: {round((1-faceDis[matchIndex])*100, 2)}%)")

        # Attendance mark karna
        markAttendance(name)

    # else:
    #     print("Unknown Person Detected")


Match Found: HARDIK_PANDYA (Confidence: 100.0%)


In [ ]:
'''
# --- 1. Take Photo Function (Colab Fixed) ---
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript(''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture Face';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    '')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

'''




In [66]:
# import cv2, numpy as np, face_recognition
# from google.colab.output import eval_js
# from base64 import b64decode

# def get_frame():
#   js = """
#     async function getFrame() {
#       if (!window.video) {
#         window.video = document.createElement('video');
#         window.video.srcObject = await navigator.mediaDevices.getUserMedia({video: true});
#         await window.video.play();
#       }
#       const canvas = document.createElement('canvas');
#       canvas.width = 640; canvas.height = 480;
#       canvas.getContext('2d').drawImage(window.video, 0, 0);
#       return canvas.toDataURL('image/jpeg', 0.8);
#     }
#   """
#   display(Javascript(js))
#   return eval_js('getFrame()')

# --- Main Logic ---
# print("System Live... (Stop karne ke liye Interrupt karein)")
# try:
#     while True:
#         # 1. Automatic frame capture
#         data = get_frame()
#         img_bytes = b64decode(data.split(',')[1])
#         img = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), 1)

#         # 2. Recognition (Small image for speed)
#         imgS = cv2.cvtColor(cv2.resize(img, (0,0), None, 0.25, 0.25), cv2.COLOR_BGR2RGB)

#         facesCurFrame = face_recognition.face_locations(imgS)
#         encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

#         for encodeFace in encodesCurFrame:
#             matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
#             faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)

#             if True in matches:
#                 matchIndex = np.argmin(faceDis)
#                 name = classNames[matchIndex].upper()
#                 print(f"Recognized: {name} ({round((1-faceDis[matchIndex])*100)}%)")
#                 markAttendance(name)

#                 recognized = True
#                 break
#             else:
#                 recognized = False

# except Exception as e:
#     print("Stopped:", e)

System Live... (Stop karne ke liye Interrupt karein)


<IPython.core.display.Javascript object>

Recognized: NIRAV_MISTRY (55%)


<IPython.core.display.Javascript object>

Recognized: NIRAV_MISTRY (56%)


...

Recognized: NIRAV_MISTRY (57%)


<IPython.core.display.Javascript object>

Recognized: NIRAV_MISTRY (57%)


<IPython.core.display.Javascript object>

KeyboardInterrupt: 